# Wind Energy


In [ ]:
import xarray as xr
import matplotlib.pyplot as plt

import sys
import os

try:
    import salientsdk as sk
except ModuleNotFoundError as e:
    if os.path.exists("../salientsdk"):
        sys.path.append(os.path.abspath(".."))
        import salientsdk as sk
    else:
        raise ModuleNotFoundError("Install salient SDK with: pip install salientsdk")

# These are the variables we need to convert wind to power:
vars = "wspd,wspd100,temp"
# Let's analyze one year of data:
start_date = "2022-01-01"
end_date = "2022-12-31"
# When we want to plot a timeseries, focus on a specific month:
plt_time = slice("2022-02-01", "2022-02-28")


sk.set_file_destination("wind_example")
sk.login("username", "password")

## Get Source Data

To start, call the Salient API to get an hourly forecast of wind-relevant meteorological varibles. Then get historical observed values for comparison.


### Set geographic bounds

The Salient SDK uses a `Location` object to specify the geographic bounds of a request. Let's analyze wind energy at 3 sites. Escalade (Texas) features modern high-capacity wind turbines on tall towers. Twin Groves (Illinois) and Windy Point (Oregon) have older moderate-capacity turbines on shorter towers.


In [ ]:
loc_file = sk.upload_location_file(
    lats=[33.735371, 40.445091, 45.710293],
    lons=[-99.646866, -88.696297, -120.848785],
    names=["Escalade", "Twin Groves I", "Windy Point IIa"],
    hub_height=[119, 80, 80],  # meters
    elevation=[444, 260, 582],  # meters
    rated_capacity=[5.6, 1.65, 2.3],  # Megawatts
    turbine_count=[65, 121, 24],
    turbine_model=["V162-5.6", "V82-1.65", "SWT-2.3-93"],
    turbine_manufacturer=["Vestas", "Vestas", "Siemens"],
    year_online=[2022, 2007, 2009],
    geoname="wind_example",
    force=False,
)
loc = sk.Location(location_file=loc_file)

### Get Historical Observed Data


In [ ]:
file_hst = sk.data_timeseries(
    loc=loc,
    variable=vars,
    field="vals",
    frequency="hourly",
    start=start_date,
    end=end_date,
    force=False,
)

# Load all of the history files into a single dataset:
hst = sk.load_multihistory(file_hst, fields=["vals"])

# Add extra columns like elevation and turbine_count to the dataset:
hst = sk.merge_location_data(hst, loc_file)
print(hst)

plt_wind100 = hst["wspd100"].sel(time=plt_time).plot(x="time", hue="location")

<xarray.Dataset> Size: 699kB
Dimensions:               (location: 3, time: 8737)
Coordinates:
  * location              (location) object 24B 'Escalade' ... 'Windy Point IIa'
  * time                  (time) datetime64[ns] 70kB 2022-01-01 ... 2022-12-31
    lat                   (location) float64 24B 33.74 40.45 45.71
    lon                   (location) float64 24B -99.65 -88.7 -120.8
Data variables:
    hub_height            (location) int64 24B 119 80 80
    elevation             (location) int64 24B 444 260 582
    rated_capacity        (location) float64 24B 5.6 1.65 2.3
    turbine_count         (location) int64 24B 65 121 24
    turbine_model         (location) object 24B 'V162-5.6' ... 'SWT-2.3-93'
    turbine_manufacturer  (location) object 24B 'Vestas' 'Vestas' 'Siemens'
    year_online           (location) int64 24B 2022 2007 2009
    wspd                  (time, location) float64 210kB 3.592 3.517 ... 1.135
    wspd100               (time, location) float64 210kB 6.842 8.0

### Downscale Forecast

The `downscale` API endpoint converts Salient's temporally granular weekly/monthly/quarterly forecasts into an hourly ensemble of timeseries.


In [ ]:
file_dsc = sk.downscale(
    loc=loc,
    variables="wspd,temp",  # RD-1483
    # variables=vars,
    date=start_date,
    # frequency="hourly", # RD-1232
    members=11,
    force=False,
)

dsc = xr.load_dataset(file_dsc)

# Add extra columns like elevation and turbine_count to the dataset:
dsc = sk.merge_location_data(dsc, loc_file)

dsc = dsc.rename({"forecast_day": "time"})

# https://salientpredictions.atlassian.net/browse/RD-1483
dsc["wspd100"] = dsc["wspd"] * (100 / 10) ** 0.25
dsc["wspd100"].attrs["long_name"] = "Wind speed at 100m"

dsc["wspd100_clim"] = dsc["wspd_clim"] * (100 / 10) ** 0.25
dsc["wspd100_clim"].attrs["long_name"] = "Wind speed at 100m"

print(dsc)

plt_wnd = (
    dsc["wspd100"]
    .sel(time=plt_time)
    .plot(
        x="time",
        hue="ensemble",
        col="location",
        size=5,
        aspect=1.0,
        add_legend=False,
        color="grey",
    )
)

for ax, location in zip(plt_wnd.axs.flat, dsc["location"].values):
    # Overlay wspd100_clim as a heavy black line on each plot
    dsc.sel(location=location, time=plt_time)["wspd100_clim"].plot(
        ax=ax, color="black", linewidth=4
    )
    # Overlay actual historical 100m wind as a blue line on each plot:
    hst.sel(location=location, time=plt_time)["wspd100"].plot(ax=ax, color="#1E90FF", linewidth=2)

<xarray.Dataset> Size: 290kB
Dimensions:               (location: 3, time: 366, ensemble: 11)
Coordinates:
  * location              (location) object 24B 'Escalade' ... 'Windy Point IIa'
  * time                  (time) datetime64[ns] 3kB 2022-01-01 ... 2023-01-01
    analog                (ensemble, time) datetime64[ns] 32kB 2022-01-01 ......
    lat                   (location) float64 24B 33.74 40.45 45.71
    lon                   (location) float64 24B -99.65 -88.7 -120.8
    forecast_date         datetime64[ns] 8B 2022-01-01
Dimensions without coordinates: ensemble
Data variables: (12/15)
    hub_height            (location) int64 24B 119 80 80
    elevation             (location) int64 24B 444 260 582
    rated_capacity        (location) float64 24B 5.6 1.65 2.3
    turbine_count         (location) int64 24B 65 121 24
    turbine_model         (location) object 24B 'V162-5.6' ... 'SWT-2.3-93'
    turbine_manufacturer  (location) object 24B 'Vestas' 'Vestas' 'Siemens'
    ...   

## Wind to Power

Now that we have wind and temperature, let's translate them to megawatts.


### Shear to Hub Height

Each of the sites has a different tower height. We need to interpolate or extrapolate 10m and 100m wind speeds to the hub height at each location.


In [ ]:
hst["wspdhh"] = sk.wind.shear_wind(hst["wspd100"], hst["wspd"], hst["hub_height"])
print(hst["wspdhh"])

plt_shear = hst.plot.scatter(x="wspd100", y="wspdhh", hue="location")
plt_diag = plt.plot([0, 25], [0, 25], "--k")

<xarray.DataArray 'wspdhh' (time: 8737, location: 3)> Size: 210kB
array([[7.18296991, 7.45286483, 1.64438469],
       [7.95819783, 6.82760911, 1.63713691],
       [8.26094285, 7.16791649, 1.10445299],
       ...,
       [4.04169768, 2.86664731, 1.98277396],
       [5.20847725, 3.18244559, 2.23860675],
       [6.92519134, 3.9305276 , 2.24932456]])
Coordinates:
  * location  (location) object 24B 'Escalade' 'Twin Groves I' 'Windy Point IIa'
  * time      (time) datetime64[ns] 70kB 2022-01-01 ... 2022-12-31
    lat       (location) float64 24B 33.74 40.45 45.71
    lon       (location) float64 24B -99.65 -88.7 -120.8
Attributes:
    long_name:   Wind Speed at hub height
    short_name:  wspdhh
    units:       m s**-1


### Correct for Air Density

Manufacturers denominate their power curves in terms of air density, often sea level. We will use elevation and temperature to modify wind speeds to match the power curve.


In [ ]:
hst["wspddc"] = sk.wind.correct_wind_density(
    wspd=hst["wspdhh"],
    dens=1.225,
    temp=hst["temp"],
    elev=hst["elevation"] + hst["hub_height"],
)

with xr.Dataset({"temp": hst.temp, "dif": hst.wspddc / hst.wspdhh}) as dif:
    dif.plot.scatter(x="temp", y="dif", hue="location")

### Define Turbine Power Curves


In [ ]:
pc = sk.wind.get_power_curve(turbine_model=hst["turbine_model"])
print(pc.head())
plt_curves = pc.plot.line(x="wind_speed")

<xarray.DataArray 'power_curve' (wind_speed: 5, turbine_model: 3)> Size: 120B
array([[0.   , 0.   , 0.   ],
       [0.027, 0.   , 0.   ],
       [0.144, 0.01 , 0.026],
       [0.289, 0.028, 0.053],
       [0.464, 0.075, 0.1  ]])
Coordinates:
  * wind_speed     (wind_speed) float64 40B 0.0 3.0 3.5 4.0 4.5
  * turbine_model  (turbine_model) object 24B 'V162-5.6' 'V82-1.65' 'SWT-2.3-93'
Attributes:
    units:          MW
    long_name:      Power
    standard_name:  power_curve


### Apply Power Curve

Feed the density-corrected hub-height wind speeds through the power curve to get power production.


In [ ]:
hst["power"] = sk.wind.calc_wind_power(hst["wspdhh"], hst["turbine_model"])
print(hst.power)

plt_power = hst["power"].sel(time=plt_time).plot.line(x="time", hue="location")

### End-to-End Wind-to-Power

As a convenience, the function `calc_wind_power_all` will shear, density correct, and calculate power. Let's use it with the `downscale` dataset so we don't have to run each step individually.


In [ ]:
pwr_dsc = sk.wind.calc_wind_power_all(dsc)
print(pwr_dsc)

# calc_wind_power_all assumes that fields are named in a standard fashion.
# If they are not, you can specify the field names explicitly:
pwr_clm = sk.wind.calc_wind_power_all(
    dsc, wspd="wspd_clim", wspd100="wspd100_clim", temp="temp_clim"
)


plt_pwr = (
    pwr_dsc["power"]
    .sel(time=plt_time)
    .plot(
        x="time",
        hue="ensemble",
        col="location",
        size=5,
        aspect=1.0,
        add_legend=False,
        color="grey",
    )
)
for ax, location in zip(plt_pwr.axs.flat, pwr_dsc["location"].values):
    pwr_clm.sel(location=location, time=plt_time)["power"].plot(ax=ax, color="black", linewidth=4)
    hst.sel(location=location, time=plt_time)["power"].plot(ax=ax, color="#1E90FF", linewidth=2)

<xarray.Dataset> Size: 228kB
Dimensions:        (location: 3, time: 366, ensemble: 11)
Coordinates:
  * location       (location) object 24B 'Escalade' ... 'Windy Point IIa'
  * time           (time) datetime64[ns] 3kB 2022-01-01 ... 2023-01-01
    analog         (ensemble, time) datetime64[ns] 32kB 2022-01-01 ... 1953-1...
    lat            (location) float64 24B 33.74 40.45 45.71
    lon            (location) float64 24B -99.65 -88.7 -120.8
    forecast_date  datetime64[ns] 8B 2022-01-01
Dimensions without coordinates: ensemble
Data variables:
    wspdhh         (ensemble, time, location) float64 97kB 16.15 8.834 ... 5.056
    power          (ensemble, time, location) float64 97kB 5.582 ... 0.1759
